# B. State purification with qubit-mediated measurement

In [1]:
# ruff: noqa
import os

os.sys.path.append("../../../..")

In [2]:
from feedback_grape.fgrape import optimize_pulse
import jax.numpy as jnp

## The cavity is initially in a  mixed state --> Goal is to purify the state

We are trying to maximize the property determined by $tr (\rho_{\text{cav}}^2)$ which is the purity

In the following, we consider an adaptive measurement
scheme, demonstrated in a series of experiments on Rydberg atoms interacting
with microwave cavities. In this scheme, the
cavity is coupled to an ancilla qubit, which can then be
read out to update our knowledge of the quantum state of
the cavity. <br> <br>
<p align="center">
    <img src="../../_static/notebook-images/b_tut.png" alt="image" width="350"/>
</p>


In [3]:
# initial state is a thermal state
n_average = 2
N_cavity = 30
# natural logarithm
beta = jnp.log((1 / n_average) + 1)
diags = jnp.exp(-beta * jnp.arange(N_cavity))
normalized_diags = diags / jnp.sum(diags, axis=0)
rho_cav = jnp.diag(normalized_diags)

### Now the thing is here, we don't need a rho_final because the purity or the reward that we want to maximize is $tr (\rho_{\text{cav}}^2)$.
Unlike fidelity expressions which wants to find how close to states are

## Next Step is to construct our POVM

In [4]:
from feedback_grape.utils.operators import cosm, sinm

### povm_measure_operator (callable): <br>
    - It should take a measurement outcome and list of params as input
    - The measurement outcome options are either 1 or -1

In [5]:
from numpy import identity
from feedback_grape.utils.operators import create, destroy
import jax


def povm_measure_operator(measurement_outcome, params):
    """
    POVM for the measurement of the cavity state.
    returns Mm ( NOT the POVM element Em = Mm_dag @ Mm ), given measurement_outcome m, gamma and delta
    """
    gamma = params[0]
    delta = params[1]
    number_operator = create(N_cavity) @ destroy(N_cavity)
    angle = (gamma * number_operator) + delta / 2 * identity(N_cavity)
    return jnp.where(
        measurement_outcome == 1,
        cosm(angle),
        sinm(angle),
    )

In [6]:
# Answer: why does RNN outputs two the same, then two the same, then two the same and so on? -> calculate during forward proagation then again during back_propagation
from feedback_grape.fgrape import Gate

measure = Gate(
    gate=povm_measure_operator,
    initial_params=jax.random.uniform(
        key=jax.random.PRNGKey(42),
        shape=(2,),
        minval=0.0,
        maxval=jnp.pi,
        dtype=jnp.float64,
    ),
    measurement_flag=True,
    # param_constraints=[
    #     [0, jnp.pi],
    #     [-2 * jnp.pi, 2 * jnp.pi],
    # ],
)

system_params = [measure]

for reward_weights in [(1,1,1,1,1),(0,0,0,0,1)]:
    result = optimize_pulse(
        U_0=rho_cav,
        C_target=None,
        system_params=system_params,
        num_time_steps=5,
        reward_weights=reward_weights,
        mode="lookup",
        goal="purity",
        max_iter=1000,
        convergence_threshold=1e-20,
        learning_rate=0.01,
        evo_type="density",
        batch_size=10,
        eval_time_steps=20,
    )

    print(f"reward weights: {reward_weights}\n final_fidelity: {result.final_fidelity}\n final_purity: {result.final_purity}\n purity_each_timestep: {jnp.mean(jnp.array(result.purity_each_timestep), axis=1)}\n")

reward weights: (1, 1, 1, 1, 1)
 final_fidelity: None
 final_purity: [0.9919538  0.99979618 0.99985247 0.84145767 0.99441463 0.99972516
 0.99979618 0.99994327 0.99994327 0.94230429]
 purity_each_timestep: [0.20000209 0.30247793 0.52649781 0.74846354 0.92343833 0.93608156
 0.93201872 0.98886305 0.98556798 0.98166295 0.98577724 0.98420409
 0.97898881 0.9876874  0.99039578 0.98847357 0.99185657 0.99063171
 0.98832074 0.98476494 0.97691869]

reward weights: (0, 0, 0, 0, 1)
 final_fidelity: None
 final_purity: [0.99769634 0.9991528  0.98077341 0.99999669 0.98575131 0.99999611
 0.9991528  0.98575131 0.99724584 0.99899433]
 purity_each_timestep: [0.20000209 0.31919734 0.42498204 0.63902323 0.75627459 0.97259339
 0.98623023 0.9879479  0.98603863 0.98877424 0.98480941 0.98982122
 0.99196245 0.99290787 0.99324596 0.99359925 0.99374561 0.99375388
 0.99375182 0.99397963 0.9944511 ]



In [7]:
from feedback_grape.utils.purity import purity

# the highest purity can be 0.995 if the initial params that initializes the lookup table
# are between 0 and pi rather than -pi and pi
print("initial purity:", purity(rho=rho_cav))
for i, state in enumerate(result.final_state):
    print(f"Purity of state {i}:", purity(rho=state))

initial purity: 0.20000208604889932
Purity of state 0: 0.9976963389616151
Purity of state 1: 0.9991528026159782
Purity of state 2: 0.9807734068777262
Purity of state 3: 0.9999966853406007
Purity of state 4: 0.9857513139831835
Purity of state 5: 0.9999961126866741
Purity of state 6: 0.9991528026159782
Purity of state 7: 0.9857513139831828
Purity of state 8: 0.9972458444854705
Purity of state 9: 0.9989943291522568


In [8]:
result.returned_params

[[Array([[1.09333296, 0.05908029],
         [1.09333296, 0.05908029],
         [1.09333296, 0.05908029],
         [1.09333296, 0.05908029],
         [1.09333296, 0.05908029],
         [1.09333296, 0.05908029],
         [1.09333296, 0.05908029],
         [1.09333296, 0.05908029],
         [1.09333296, 0.05908029],
         [1.09333296, 0.05908029]], dtype=float64)],
 [Array([[1.09323741e+00, 4.51206584e-01],
         [1.57079633e+00, 8.24752039e-17],
         [1.09323741e+00, 4.51206584e-01],
         [1.57079633e+00, 8.24752039e-17],
         [1.09323741e+00, 4.51206584e-01],
         [1.57079633e+00, 8.24752039e-17],
         [1.57079633e+00, 8.24752039e-17],
         [1.09323741e+00, 4.51206584e-01],
         [1.09323741e+00, 4.51206584e-01],
         [1.57079633e+00, 8.24752039e-17]], dtype=float64)],
 [Array([[ 1.57079633e+00,  2.97152495e-15],
         [ 1.02955772e+00, -8.74809684e-03],
         [ 1.57079633e+00,  2.97152495e-15],
         [ 1.40156193e+00,  7.14115220e-01],
    